# ID2222 Data Mining - Homework 2

Notebook by Beatrice Insalata, Laura Puccioni

For this task, we are asked to implement the A-Priori algorithm to identify frequent itemsets with a support of at least 's' in a sales transactions dataset. We have used the Python language to evaluate and validate the A-Priori implementation on the given dataset.

## Dataset
Our dataset consists of a sale transaction database (T10I4D100K). The dataset contains 100,000 baskets summing up to 1,010,228 items, with a market size of 870 different items.

In [9]:
#Here we read from a file .dat assuming that on every row of the file there is a basket of items and create a list of baskets

with open('T10I4D100K.dat', "r") as f:
    baskets = list(
        map(
            lambda basket: [int(item_id) for item_id in basket.split()],
            f.read().splitlines()
        )
    )
f.close()

# A-Priori Algorithm Implementation

The Apriori algorithm operates in two passes. In the initial pass, it scans through the baskets, counting the occurrences of each individual item (singletons). In the subsequent pass, the algorithm revisits the baskets, focusing only on pairs where both elements are identified as frequent during the first pass. This process continues iteratively, incrementing the itemset size with each pass. The primary aim is to efficiently identify and prune non-frequent itemsets, ultimately discovering frequent itemsets and their support levels in the dataset.

The iterative nature of the algorithm allows it to progressively explore larger itemsets while leveraging the knowledge gained from previous passes. This two-pass strategy minimizes the need for multiple scans of the entire dataset and optimizes the discovery of frequent itemsets.

## Pass 1 : Find frequent singletons

During the initial pass, the Apriori algorithm identifies frequent single elements, or 1-itemsets, within the dataset. The term "frequent" here implies that these individual elements occur with a frequency at least equal to a specified threshold called "support". Essentially, this step involves finding and recording items that are individually popular or common, laying the foundation for subsequent steps in discovering larger sets of items with significant occurrences.

In [10]:
"""
This function takes as arguments the set of baskets and the threshold called "support", 
and returns the set of frequent singletons, that is the frequent 1-item sets.
"""
def find_frequent_singletons_with_support(baskets, support):
    
    occurrences = {} # a dictionary containing occurence of items in baskets
    for itemset in baskets:
        for item in itemset:
            if item not in occurrences: # this item didn't exist in previous baskets, initialize with 1
                occurrences[item] = 1
            else:
                occurrences[item] += 1
    #print({key:value for key, value in occurrences.items() if value >= support})
    return {key for key, value in occurrences.items() if value >= support}

We can visualize how the step 1 works on a small set of items, for example using the first 5 baskets of the given dataset:

In [11]:
example_dataset = baskets[0:5]
print(example_dataset)

[[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834], [39, 120, 124, 205, 401, 581, 704, 814, 825, 834], [35, 249, 674, 712, 733, 759, 854, 950], [39, 422, 449, 704, 825, 857, 895, 937, 954, 964], [15, 229, 262, 283, 294, 352, 381, 708, 738, 766, 853, 883, 966, 978]]


If, for instance, we want support = 2, we keep the elements that appear in the dataset at least 2 times. We can do this by hand and then check the correctness of the previous function. In this case 39 appears 2 times, 704 appears 2 times, 825 appears 3 times and 834 appears 2 times. So in the end we will have a list of singleton of this type: {39, 704, 825, 934}.
Let's check this by calling the find_frequent_singletons_with_support function on the example_dataset with support = 2:

In [12]:
find_frequent_singletons_with_support(example_dataset, 2)

{825: 3, 834: 2, 39: 2, 704: 2}


{39, 704, 825, 834}

Devo finire da qui in poi, aggiungerò più esempi e spiegazioni

In [13]:
def generate_candidates(prev_frequent_itemsets, k):
    candidates = set()
    
    # Ensure prev_frequent_itemsets only contains iterable elements
    prev_frequent_itemsets = [itemset if isinstance(itemset, (list, tuple)) else [itemset] for itemset in prev_frequent_itemsets]

    for i in range(len(prev_frequent_itemsets)):
        for j in range(i + 1, len(prev_frequent_itemsets)):
            union_set = set(prev_frequent_itemsets[i]) | set(prev_frequent_itemsets[j])
            if len(union_set) == k + 1:
                candidates.add(tuple(sorted(union_set)))
    
    return list(candidates)

newset = generate_candidates(find_frequent_singletons_with_support(example_dataset, 2), 1)
print(newset)
generate_candidates(newset, 2)

{825: 3, 834: 2, 39: 2, 704: 2}
[(39, 834), (704, 825), (39, 704), (704, 834), (39, 825), (825, 834)]


[(39, 704, 834), (39, 704, 825), (704, 825, 834), (39, 825, 834)]

In [14]:
def apriori(baskets, min_support):
    frequent_itemsets = []
    k = 0

    # Find frequent 1-itemsets
    frequent_singletons = find_frequent_singletons_with_support(baskets, min_support)
    frequent_itemsets.extend(frequent_singletons)

    while frequent_itemsets: #loop until there are no more frequent itemsets
        k += 1
        # Generate candidate itemsets
        candidate_itemsets = generate_candidates(frequent_itemsets, k)

        # Count occurrences of candidate itemsets
        itemset_counts = {itemset: 0 for itemset in candidate_itemsets}
        for basket in baskets:
            for itemset in itemset_counts:
                if set(itemset).issubset(basket):
                    itemset_counts[itemset] += 1

        # Filter frequent itemsets
        frequent_itemsets = [itemset for itemset, count in itemset_counts.items() if count >= min_support]

        # Print frequent itemsets of size k
        print(f"Frequent {k}-itemsets:", frequent_itemsets)
        
apriori(baskets[0:1000], 12)


{25: 13, 52: 19, 240: 15, 274: 33, 368: 80, 448: 13, 538: 42, 561: 33, 775: 33, 825: 25, 39: 48, 120: 56, 205: 28, 401: 31, 581: 28, 704: 18, 814: 19, 35: 19, 674: 24, 733: 16, 854: 36, 950: 19, 449: 20, 895: 46, 937: 45, 964: 19, 229: 20, 283: 57, 294: 12, 381: 31, 738: 26, 766: 54, 853: 19, 883: 45, 966: 36, 978: 12, 143: 14, 569: 38, 620: 17, 798: 34, 214: 14, 350: 32, 529: 56, 658: 22, 682: 35, 782: 37, 809: 20, 947: 28, 970: 26, 227: 16, 390: 27, 71: 49, 192: 21, 279: 32, 280: 21, 496: 15, 530: 12, 597: 30, 675: 32, 720: 40, 914: 36, 932: 19, 183: 38, 193: 12, 217: 67, 256: 13, 276: 27, 653: 29, 706: 17, 878: 27, 161: 24, 175: 25, 177: 60, 424: 19, 571: 31, 623: 24, 795: 38, 910: 16, 960: 24, 125: 16, 130: 16, 392: 26, 461: 15, 801: 12, 862: 40, 27: 26, 78: 23, 921: 32, 147: 20, 411: 21, 572: 19, 579: 28, 778: 35, 803: 26, 903: 12, 266: 16, 523: 27, 614: 27, 888: 30, 944: 24, 43: 23, 70: 25, 204: 34, 334: 16, 480: 28, 874: 27, 151: 32, 830: 13, 890: 14, 73: 20, 118: 14, 310: 20, 4